In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/Titanic.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M3/CLASE_16_Regresion_Lineal_Multiple/Data/Titanic.csv?token=AA4GFHOI7CQ5RN3JDHOQCF263APRM    
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# Dummies

### Introducción
Las variables ficticias o "dummies" a menudo se usan en conjuntos de datos para ayudar a poner números en objetos categóricos. Muchas veces, cuando los científicos de datos analizan datos, se les asigna la tarea de descomponer los datos en objetos que puedan modelarse. Aunque es posible modelar objetos escritos, hay muchos objetos que solo se pueden modelar con números.

Como futuros científicos de datos es fundamental  crear números a partir de valores categóricos de forma correcta.

Al momento de entrenar un modelo de machine learning es necesario transformar y codificar las variables de tipo categoricas. Existen dos tipos de variables categoricas: 
* Nominal. Una variable puede ser tratada como nominal cuando sus valores representan categorías que no obedecen a una clasificación intrínseca. Por ejemplo, el departamento de la compañía en el que trabaja un empleado. Algunos ejemplos de variables nominales son: región, código postal o confesión religiosa.

* Ordinal. Una variable puede ser tratada como ordinal cuando sus valores representan categorías con alguna clasificación intrínseca. Por ejemplo, los niveles de satisfacción con un servicio, que abarquen desde muy insatisfecho hasta muy satisfecho. Entre los ejemplos de variables ordinales se incluyen escalas de actitud que representan el grado de satisfacción o confianza y las puntuaciones de evaluación de las preferencias.

[<img src="img/tipo_de_variables.png" width="400" height="200" align='center'>](http://digitalhouse.com.ar/)



Aprenderemos a codificar ambas clases en la presente notebook.
### Acerca del dataset.
Comenzaremos  con la carga un dataset de Titanic. Este es un conjunto de datos muy pequeño que consiste en datos de variables demográficas de los pasajeros de la emblematica embarcacion, clasificados por género, rango de tarifa del ticket, etc.


## Variables Categóricas Nominales:

In [ ]:
import pandas as pd

# cargamos el dataset 
df = df= pd.read_csv('../Data/Titanic.csv')

In [ ]:
df.head()

Podemos obtener la/las variables dummies para la columna "sex" donde la misma solo puede tener dos valores posibles: male o female. Las variables dummies obtenidas representan el estado de la variable categórica (presencia/ausencia) de la misma. Utilizando la función get_dummies de pandas:

### Método 1. Pandas get_dummies

In [ ]:
dummy = pd.get_dummies(df['Sex'])
dummy.head()

In [ ]:
# concatenamos ambos datasets 
df = pd.concat([df,dummy],axis=1)
# 
df['Survived']=df.Survived.apply(lambda x: 'yes' if x==1 else 'No')
df.head()

In [ ]:
df.head()

### Método 2. sklearn.preprocessing.OneHotEncoder

¿ sklearn.preprocessing.OneHotEncoder o de pd.get_dummies ? 
Sklearn.preprocessing.OneHotEncoder devuelve un objeto de la clase sklearn.preprocessing.OneHotEncoder. Podemos ajustar este objeto en el conjunto de entrenamiento y luego usar el mismo objeto para transformar el conjunto de prueba. Por otro lado, pd.get_dummies devuelve "dataframe" con codificaciones basadas en los valores en el "dataframe" que le pasamos. Esto podría ser bueno para un análisis rápido, pero para un proyecto que luego tendriamos que poner en producción es mejor tener el objeto OneHotEncoder para luego poder "transformar" nuevos datos que lleguen a nuestro servidor.

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
# fiteo y transformo la columna "sex"
dummy_oneHot = onehot_encoder.fit_transform(df[['Sex']])
# pongo un vector en un dataset.
dummy_oneHot = pd.DataFrame(dummy_oneHot.toarray(),columns=df['Sex'].unique())
dummy_oneHot.head()

### Variables dummies y multicolinealidad.
La "trampa" de variable dummy se manifiesta directamente a partir de una codificación "One-hot-encoding" aplicada en variables categóricas. Como se discutió anteriormente (supuestos de regresion lineal), el tamaño de los vectores de un "one-Hot-encoding" es igual al número de valores únicos que una columna categórica puede tomar. Imaginemos la variable speed con 3 niveles: 


$speed_{low} + speed_{Medium} + speed_{High} = 1$


De la relación anterior, podemos expresar cualquiera de las tres variables independientes en términos de las otras dos, tomemos $speed_l{low}$ y expresarla en términos de $speed_{medium}$ y $speed_{high}$:


$speed_{low} = 1 - speed_{Medium} - speed_{High} = 1$

### ¿Porque es malo?

Como discutimos anteriormente, esto tiene una implicancia directa en el problema de regresión lineal. 


Podemos sustituir speed_low en la ecuación-1 con su valor en la ecuación-2. Esto en realidad significa que (al menos) una de las características con las que estamos trabajando es **redundante**; esa característica podría ser cualquiera de las tres, ya que la ecuación-2 podría escribirse con cualquiera de ellas. Por lo tanto, estamos haciendo que nuestro modelo aprenda un peso adicional que realmente no es necesario. Esto consume potencia computacional y tiempo. Esto también proporciona un objetivo de optimización que podría no ser muy razonable y también podría ser difícil trabajar con él. 

### ¿Cómo lo solucionamos?
Como la codificación "One-Hot-Enconding" induce una multicolinealidad perfecta, eliminamos una de las columnas de las características codificadas. Por ejemplo, podemos elegir dropear $speed_{medium}$ o en nuestro ejemplo "male" aunque la elección es completamente arbitraria.


* Cuando usamos pd.get_dummies, podemos pasar un argumento adicional, drop_first = True para descartar la primera columna nueva que obtenemos después de la codificación.

In [ ]:
dummy_correct = pd.get_dummies(df['Sex'],drop_first=True)
dummy_correct.head()

* Cuando usamos sklearn.preprocessing.OneHotEncoder y queremos dropear una de las nuevas columnas, pasamos el argumento drop = 'first' constructor de la clase OneHotEncoder.

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder2 = OneHotEncoder(drop='first')
# fiteo y transformo la columna "sex"
dummy_oneHot_correct = onehot_encoder2.fit_transform(df[['Sex']])
# pongo un vector en un dataset.
dummy_oneHot_correct = pd.DataFrame(dummy_oneHot_correct.toarray())
dummy_oneHot_correct.head()

## Varibles categoricas Ordinales
Utilizaremos la variables "Survived" para generar una variable derivada que tenga en cuenta que si sobrevivio es "mejor" que si no lo hizo. Con esto podemos generar por ejemplo una variable categórica que puede ser utilizada como vector "objetivo" en un problema de clasificación.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(df['Survived'])

### Conclusiones:
Al momento de entrenar un modelo de machine learning es necesario transformar y codificar las variables de tipo categóricas. 

<div id="caja8" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_saber_mas.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label></label></div>
</div>


Recursos extra: 
Más recursos acerca de ["Encoders"](https://towardsdatascience.com/benchmarking-categorical-encoders-9c322bd77ee8)